In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from datetime import datetime
from train import *
from loss import *
from accuracy import *
from model import *

import cifar_input
import os.path
import time
import numpy as np
import tensorflow as tf

In [2]:
max_steps = 500000
train_dir = 'cifar10_vgg_model2/'
batch_size = 128
log_device_placement = False

In [3]:
with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()
    
    # Generating images and its labels
    # build_inputs('cifar10/cifar100', cifar dataset dir, batch size, mode)
    images, labels = cifar_input.build_input('cifar10', '../../cifar/cifar10/data_batch*', batch_size, 'train')
    
    # Creating graph. NUM_CLASESS=10 (CIFAR-10) or NUM_CLASESS=100 (CIFAR-100)
    logits = inference(images, NUM_CLASSES=10)
    
    # Loss/Error and Accuracy
    losses = loss(logits, labels)
    accuracies = accuracy(logits, labels)
    
    # Our train_op (Only minimizing loss)
    train_op = train(losses, global_step, batch_size)
    
    
    # SessionRunHook. Logging will be done each x steps.
    class _LoggerHook(tf.train.SessionRunHook):
        
      def begin(self):
        self._step = -1
        # Creating train_dir if it does not exist and writing to log file
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)
        open(train_dir+'training_data.csv', 'w').close()
        f = open(train_dir+"log.txt",'ab')
        f.write('\n\n==== Run ===\nInfo: VGG\n')
        f.close()

      def before_run(self, run_context):
        # Increment step, reset start_time, and asking for loss and accuracy tensor
        self._step += 1
        self._start_time = time.time()
        return tf.train.SessionRunArgs([losses, accuracies])

      def after_run(self, run_context, run_values):
        duration = time.time() - self._start_time # Calculating time
        loss_value = run_values.results[0]
        accuracy_value = run_values.results[1]
        # Printing log, accuracy, and loss
        if self._step % 10 == 0:
          print("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
                
          f = open(train_dir+"log.txt",'ab')
          f.write("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
          f.close()
                
          f = open(train_dir+"training_data.csv",'ab')
          f.write('{0},{1},{2}\n'.format(self._step, loss_value, accuracy_value))
          f.close()
            
    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
                                           hooks=[tf.train.StopAtStepHook(last_step=max_steps),
                                                  tf.train.NanTensorHook(losses),
                                                  _LoggerHook()],save_checkpoint_secs=30, 
                                           config=tf.ConfigProto(
                                               log_device_placement=log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into cifar10_vgg_model2/model.ckpt.
2017-03-16 13:35:52.853589: step 0, error = 4.8358, accuracy = 0.0859. (3.472 sec/step)

2017-03-16 13:35:55.684678: step 10, error = 5.1074, accuracy = 0.2031. (0.336 sec/step)

2017-03-16 13:35:59.168293: step 20, error = 5.4176, accuracy = 0.1562. (0.333 sec/step)

2017-03-16 13:36:03.397333: step 30, error = 5.1448, accuracy = 0.1641. (0.438 sec/step)

2017-03-16 13:36:07.534616: step 40, error = 4.9131, accuracy = 0.1953. (0.434 sec/step)

2017-03-16 13:36:11.655467: step 50, error = 4.9711, accuracy = 0.2266. (0.398 sec/step)

2017-03-16 13:36:15.932149: step 60, error = 4.6127, accuracy = 0.2812. (0.436 sec/step)

2017-03-16 13:36:20.097891: step 70, error = 4.5922, accuracy = 0.2891. (0.358 sec/step)

INFO:tensorflow:Saving checkpoints for 78 into cifar10_vgg_model2/model.ckpt.
2017-03-16 13:36:23.644583: step 80, error = 4.7797, accuracy = 0.2812. (0.217 sec

ResourceExhaustedError: cifar10_vgg_model2/model.ckpt-28172_temp_37ef64c60f354a74a63a9b5eb379e40f/part-00001-of-00002.data-00000-of-00001.tempstate8362418674437790013
	 [[Node: save/SaveV2_1 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](save/ShardedFilename_1, save/SaveV2_1/tensor_names, save/SaveV2_1/shape_and_slices, conv1_1/biases, conv1_1/biases/ExponentialMovingAverage, conv1_1/weights, conv1_1/weights/ExponentialMovingAverage, conv1_2/biases, conv1_2/biases/ExponentialMovingAverage, conv1_2/weights, conv1_2/weights/ExponentialMovingAverage, conv2_1/biases, conv2_1/bi